In [1]:
import sys
from pathlib import Path
import numpy as np
from scipy.ndimage import center_of_mass
#import warnings
#warnings.filterwarnings("error")
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import numpy as np
from superpose3d import Superpose3D
PIPELINE_ROOT = Path('../src').resolve().parent.parent
sys.path.append(PIPELINE_ROOT.as_posix())
print(PIPELINE_ROOT)

from library.registration.brain_structure_manager import BrainStructureManager
from library.registration.algorithm import umeyama

/home/eddyod/programming/preprocessing-pipeline/src


In [2]:
def brain_to_atlas_transform(brain_coord, r, t):
    brain_coord = np.array(brain_coord).reshape(3, 1) # Convert to a column vector
    atlas_coord = r @ brain_coord + t
    return atlas_coord.T[0] # Convert back to a row vector

def calculate_distance(com1, com2):
    return (np.linalg.norm(com1 - com2))

def plot_point_sets_3d(point_sets):
    df = pd.DataFrame()
    for data, label in point_sets:
        df_cur = pd.DataFrame(data.T, columns=['x', 'y', 'z'])
        df_cur['label'] = label
        #df = df.concat(df_cur, ignore_index=True)
        df = pd.concat([df, df_cur], axis= 0)
    
    fig = px.scatter_3d(df, x='x', y='y', z='z', color='label')
    return fig

In [3]:
animal = 'DK55'
brain = BrainStructureManager(animal)
brain.fixed_brain = BrainStructureManager('Allen')

In [4]:
moving_coms = brain.get_coms(annotator_id=2)
allen_coms = brain.fixed_brain.get_coms(annotator_id=1)
common_keys = allen_coms.keys() & moving_coms.keys()
brain_regions = sorted(moving_coms.keys())
allen_points = np.array([allen_coms[s] for s in brain_regions if s in common_keys])
moving_points = np.array([moving_coms[s] for s in brain_regions if s in common_keys])

In [5]:
allen_point_dict = {s:allen_coms[s] for s in brain_regions if s in common_keys}
moving_point_dict = {s:moving_coms[s] for s in brain_regions if s in common_keys}

In [6]:
moving_point_dict

{'3N_L': [11207.5, 5005.76, 4620.0],
 '3N_R': [11172.8, 4960.67, 4900.0],
 '4N_L': [11525.2, 5043.96, 4460.0],
 '4N_R': [11474.0, 4986.83, 4940.0],
 '5N_L': [12184.6, 6245.08, 3400.0],
 '5N_R': [12198.9, 6011.09, 6320.0],
 '6N_L': [12747.7, 6215.66, 4500.0],
 '6N_R': [12691.9, 6174.66, 5240.0],
 '7N_L': [13080.7, 7323.6, 3660.0],
 '7N_R': [12927.3, 7183.8, 6300.0],
 '7n_L': [12650.95, 6181.5, 4020.0],
 'AP': [14439.6, 5705.57, 4920.0],
 'Amb_L': [13717.5, 7135.54, 3660.0],
 'Amb_R': [13785.2, 6869.11, 6340.0],
 'DC_L': [13327.7, 5646.89, 2660.0],
 'DC_R': [13334.3, 5283.99, 7080.0],
 'LC_L': [12584.0, 5309.77, 3960.0],
 'LC_R': [12679.9, 5218.19, 5680.0],
 'LRt_L': [14545.3, 7358.92, 3680.0],
 'LRt_R': [14848.1, 7225.04, 6280.0],
 'PBG_L': [11314.1, 5133.68, 2860.0],
 'PBG_R': [11309.0, 4916.5, 6740.0]}

In [ ]:
allen_coms['SC']

In [7]:
distances = []
sortme = {}
for structure in common_keys:
    (x,y,z) = allen_point_dict[structure]
    allen_point = np.array([x,y,z])    
    moving_point = np.array(moving_point_dict[structure])
    #print(atlas_point, allen_point)
    d = calculate_distance(allen_point, moving_point)
    distances.append(d)
    sortme[structure] = d
print(f'n={len(distances)}, min={min(distances)} max={max(distances)}, mean={np.mean(distances)}')
ds = {k: v for k, v in sorted(sortme.items(), key=lambda item: item[1])}
for structure, d in ds.items():
    print(f'{structure} distance from Allen={round(d,2)} micrometers')

n=22, min=2018.2507510465596 max=2614.012388589618, mean=2339.176779799585
Amb_L distance from Allen=2018.25 micrometers
Amb_R distance from Allen=2051.76 micrometers
AP distance from Allen=2066.51 micrometers
7n_L distance from Allen=2194.52 micrometers
DC_L distance from Allen=2229.95 micrometers
7N_R distance from Allen=2242.01 micrometers
LC_L distance from Allen=2274.63 micrometers
6N_R distance from Allen=2310.7 micrometers
5N_L distance from Allen=2315.92 micrometers
LRt_L distance from Allen=2342.58 micrometers
5N_R distance from Allen=2344.08 micrometers
DC_R distance from Allen=2344.28 micrometers
6N_L distance from Allen=2352.38 micrometers
LC_R distance from Allen=2389.35 micrometers
7N_L distance from Allen=2393.08 micrometers
PBG_L distance from Allen=2406.82 micrometers
PBG_R distance from Allen=2445.87 micrometers
4N_R distance from Allen=2461.0 micrometers
4N_L distance from Allen=2509.84 micrometers
3N_R distance from Allen=2563.92 micrometers
3N_L distance from Allen

In [ ]:
plot_point_sets_3d([
    (atlas_points.T, 'unaligned moving centers'),
    (allen_points.T, 'Allen centers')
])

In [16]:
(RMSD, _, _, _) = Superpose3D(allen_points, moving_points, allow_rescale=True)
print(RMSD)

221.02443674967975


In [17]:
R, t = umeyama(moving_points.T, allen_points.T)

In [18]:
print(R)
print(t)

[[ 1.07762522 -0.19058936 -0.00973869]
 [ 0.19077436  1.07442001  0.08319884]
 [-0.00492819 -0.08362178  1.0911821 ]]
[[-1811.87371515]
 [-4065.73917681]
 [  940.3294948 ]]


In [19]:
reg_points = R @ moving_points.T + t

In [20]:
print('Mean')
print(np.mean(allen_points, axis=0))
print(np.mean(reg_points, axis=1))
print('Min')
print(np.min(allen_points, axis=0))
print(np.min(reg_points, axis=1))
print('Max')
print(np.max(allen_points, axis=0))
print(np.max(reg_points, axis=1))

Mean
[10707.85727273  5166.11227273  5647.64363636]
[10707.85727273  5166.11227273  5647.64363636]
Min
[9102.49 3774.79 3229.  ]
[9235.04689181 3803.26206037 3304.9894971 ]
Max
[12653.24  6990.91  8156.39]
[12750.63866088  7052.11391658  8158.3281967 ]


In [22]:
reg_point_dict = {s:moving_coms[s] for s in brain_regions if s in common_keys}

In [23]:
distances = []
sortme = {}
for structure in common_keys:
    (x,y,z) = allen_point_dict[structure]
    allen_point = np.array([x,y,z])    
    moving_point = np.array(moving_point_dict[structure])
    reg_point = brain_to_atlas_transform(moving_point, R, t)
    d = calculate_distance(allen_point, reg_point)
    distances.append(d)
    #print(f'{structure} COM={reg_point} distance={round(d,2)}')
    sortme[structure] = d

print(f'n={len(distances)}, min={min(distances)} max={max(distances)}, mean={np.mean(distances)}')
ds = {k: v for k, v in sorted(sortme.items(), key=lambda item: item[1])}
for structure, d in ds.items():
    print(f'{structure} distance from Allen={round(d,2)} micrometers')

n=22, min=38.11424881491477 max=429.1977924102672, mean=200.18559492616384
PBG_L distance from Allen=38.11 micrometers
5N_R distance from Allen=79.9 micrometers
PBG_R distance from Allen=93.17 micrometers
LC_L distance from Allen=94.98 micrometers
5N_L distance from Allen=106.56 micrometers
3N_R distance from Allen=140.14 micrometers
4N_R distance from Allen=154.89 micrometers
LRt_L distance from Allen=162.58 micrometers
3N_L distance from Allen=169.57 micrometers
4N_L distance from Allen=179.51 micrometers
LC_R distance from Allen=179.99 micrometers
7n_L distance from Allen=184.27 micrometers
7N_L distance from Allen=191.43 micrometers
6N_L distance from Allen=218.37 micrometers
AP distance from Allen=232.01 micrometers
7N_R distance from Allen=235.61 micrometers
6N_R distance from Allen=250.51 micrometers
Amb_R distance from Allen=297.62 micrometers
DC_L distance from Allen=305.75 micrometers
Amb_L distance from Allen=327.21 micrometers
DC_R distance from Allen=332.7 micrometers
LRt_

In [ ]:
# superpose, allow_rescale=False 34 279 105 613
# superpoase with scaling 34 271 74 621
# umeyama with scaling 34 271 74 621

In [ ]:
plot_point_sets_3d([
    (reg_points, 'registered moving centers'),
    (allen_points.T, 'Allen centers')    
])